In [12]:
import json
import configparser
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

# 設定
config = configparser.ConfigParser()
print(config)
# config.read('/mnt/d/Python/config/config_V20.txt') # パスの指定が必要です
config.read(r"../config/config_V20.txt")
print(config.sections())
accountID = config['oanda']['account_id']
api_key = config['oanda']['api_key']

api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
params = {
    'count': 3,
    'granularity': 'H1',
}
r = instruments.InstrumentsCandles(instrument='USD_JPY', params=params)
response = api.request(r)
print(json.dumps(response, indent=2))

['oanda']
{
  "instrument": "USD_JPY",
  "granularity": "H1",
  "candles": [
    {
      "complete": true,
      "volume": 101,
      "time": "1580162400.000000000",
      "mid": {
        "o": "108.891",
        "h": "108.918",
        "l": "108.884",
        "c": "108.917"
      }
    },
    {
      "complete": true,
      "volume": 1126,
      "time": "1580166000.000000000",
      "mid": {
        "o": "108.916",
        "h": "108.964",
        "l": "108.898",
        "c": "108.932"
      }
    },
    {
      "complete": false,
      "volume": 1208,
      "time": "1580169600.000000000",
      "mid": {
        "o": "108.932",
        "h": "108.966",
        "l": "108.920",
        "c": "108.938"
      }
    }
  ]
}


In [13]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
from datetime import datetime
# start～endまでのデータ取得
def get_period_data(start, end, minute, instrument='USD_JPY'):
    timestamp = start.timestamp()
    concats = []
    count = 5000
    while True:
        df, last_timestamp = send_api(count, timestamp, minute, instrument)
        concats.append(df)
        print(df)
        if last_timestamp > end.timestamp() or len(df) < count:
            break
        timestamp = last_timestamp + (60 * minute)
    df = pd.concat(concats)
    if end is None:
        print(df)
        return df
    else:
        return df[df.index < end]
# 時間足のdfを取得
def send_api(count, start, minute, instrument):
    # oandaへのリクエストの送信
    #api_key = '取得したアクセストークンを入れてください'
    api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
    if minute == 1/60:
        granularity = 'S5'
    elif minute == 1:
        granularity = 'M1'
    elif minute == 5:
        granularity = 'M5'
    elif minute == 15:
        granularity = 'M15'
    elif minute == 60:
        granularity = 'H1'
    params = {
        'count': count,
        'granularity': granularity,
    }
    if start is not None:
        params['from'] = start
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(r)

    # レスポンスの整形
    def join_json(candle):
        tmp = candle['mid']
        tmp['time'] = candle['time']
        tmp['v'] = candle['volume']
        tmp['complete'] = candle['complete']
        return tmp
    data_list = [join_json(candle) for candle in response['candles']]
    df = pd.DataFrame(data_list)
    last_timestamp = int(float(df.iloc[-1]['time']))
    
    # 型変更
    df['time'] = df['time'].astype('float64')
    df['o'] = df['o'].astype('float64')
    df['h'] = df['h'].astype('float64')
    df['l'] = df['l'].astype('float64')
    df['c'] = df['c'].astype('float64')
    df['v'] = df['v'].astype('float64')

    # タイムゾーンの変更、インデックス化
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'] + pd.Timedelta('09:00:00') # 日本時間へ変換
    df.set_index('time', inplace=True)                 # 時間をインデックスにする
    df = df.loc[:,['o','h','l', 'c', 'v', 'complete']] # 列の順番変更
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'amount'})

    return df, last_timestamp
minute = 1
start = datetime.strptime('2019-01-28 09:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.now()
#end = datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
df = get_period_data(start, end, minute, instrument='USD_JPY')
df

                        open     high      low    close  amount  complete
time                                                                     
2019-01-28 09:00:00  109.416  109.430  109.382  109.412    45.0      True
2019-01-28 09:01:00  109.414  109.428  109.400  109.428    23.0      True
2019-01-28 09:02:00  109.426  109.426  109.396  109.419    30.0      True
2019-01-28 09:03:00  109.422  109.424  109.420  109.424    10.0      True
2019-01-28 09:04:00  109.422  109.424  109.414  109.419     8.0      True
...                      ...      ...      ...      ...     ...       ...
2019-01-31 23:33:00  108.553  108.553  108.532  108.537    32.0      True
2019-01-31 23:34:00  108.535  108.539  108.528  108.536    12.0      True
2019-01-31 23:35:00  108.538  108.560  108.536  108.556    28.0      True
2019-01-31 23:36:00  108.553  108.558  108.520  108.520    38.0      True
2019-01-31 23:37:00  108.522  108.534  108.520  108.523    35.0      True

[5000 rows x 6 columns]
             

                        open     high      low    close  amount  complete
time                                                                     
2019-03-14 17:35:00  111.678  111.692  111.678  111.690    10.0      True
2019-03-14 17:36:00  111.687  111.688  111.676  111.686    14.0      True
2019-03-14 17:37:00  111.688  111.692  111.682  111.682    15.0      True
2019-03-14 17:38:00  111.680  111.688  111.678  111.680    11.0      True
2019-03-14 17:39:00  111.683  111.692  111.683  111.687     7.0      True
...                      ...      ...      ...      ...     ...       ...
2019-03-20 13:17:00  111.605  111.605  111.598  111.598     4.0      True
2019-03-20 13:18:00  111.595  111.595  111.595  111.595     1.0      True
2019-03-20 13:19:00  111.597  111.600  111.597  111.598     3.0      True
2019-03-20 13:20:00  111.600  111.600  111.598  111.598     6.0      True
2019-03-20 13:21:00  111.600  111.602  111.600  111.602     2.0      True

[5000 rows x 6 columns]
             

                        open     high      low    close  amount  complete
time                                                                     
2019-05-02 12:53:00  111.545  111.545  111.538  111.545     7.0      True
2019-05-02 12:54:00  111.542  111.542  111.542  111.542     1.0      True
2019-05-02 12:57:00  111.546  111.548  111.546  111.548     2.0      True
2019-05-02 13:01:00  111.550  111.550  111.544  111.548     6.0      True
2019-05-02 13:08:00  111.552  111.552  111.550  111.550     2.0      True
...                      ...      ...      ...      ...     ...       ...
2019-05-08 07:44:00  110.239  110.268  110.239  110.268    19.0      True
2019-05-08 07:45:00  110.266  110.268  110.266  110.268     4.0      True
2019-05-08 07:46:00  110.266  110.278  110.266  110.271    11.0      True
2019-05-08 07:47:00  110.268  110.268  110.264  110.266     4.0      True
2019-05-08 07:48:00  110.268  110.270  110.268  110.268     3.0      True

[5000 rows x 6 columns]
             

                        open     high      low    close  amount  complete
time                                                                     
2019-06-18 20:50:00  108.270  108.286  108.268  108.283    14.0      True
2019-06-18 20:51:00  108.286  108.287  108.282  108.287     6.0      True
2019-06-18 20:52:00  108.290  108.290  108.283  108.286     5.0      True
2019-06-18 20:53:00  108.283  108.285  108.275  108.275    10.0      True
2019-06-18 20:54:00  108.273  108.278  108.272  108.274     9.0      True
...                      ...      ...      ...      ...     ...       ...
2019-06-24 11:26:00  107.387  107.387  107.384  107.387     3.0      True
2019-06-24 11:27:00  107.389  107.392  107.380  107.380     8.0      True
2019-06-24 11:28:00  107.378  107.380  107.374  107.374     4.0      True
2019-06-24 11:29:00  107.378  107.380  107.374  107.374     8.0      True
2019-06-24 11:30:00  107.372  107.373  107.366  107.371     7.0      True

[5000 rows x 6 columns]
             

                        open     high      low    close  amount  complete
time                                                                     
2019-08-02 22:07:00  107.014  107.036  106.994  107.009   125.0      True
2019-08-02 22:08:00  107.012  107.040  107.002  107.008   102.0      True
2019-08-02 22:09:00  107.011  107.062  106.999  107.020   190.0      True
2019-08-02 22:10:00  107.023  107.030  106.954  106.964   179.0      True
2019-08-02 22:11:00  106.962  106.980  106.949  106.974   102.0      True
...                      ...      ...      ...      ...     ...       ...
2019-08-08 11:27:00  106.218  106.233  106.214  106.228    28.0      True
2019-08-08 11:28:00  106.224  106.232  106.224  106.227    12.0      True
2019-08-08 11:29:00  106.225  106.233  106.225  106.228    14.0      True
2019-08-08 11:30:00  106.226  106.226  106.204  106.206    14.0      True
2019-08-08 11:31:00  106.209  106.212  106.186  106.186    30.0      True

[5000 rows x 6 columns]
             

                        open     high      low    close  amount  complete
time                                                                     
2019-09-18 09:53:00  108.165  108.191  108.154  108.184    29.0      True
2019-09-18 09:54:00  108.182  108.191  108.177  108.186    34.0      True
2019-09-18 09:55:00  108.188  108.192  108.176  108.188    35.0      True
2019-09-18 09:56:00  108.186  108.186  108.178  108.180     9.0      True
2019-09-18 09:57:00  108.178  108.185  108.178  108.185     4.0      True
...                      ...      ...      ...      ...     ...       ...
2019-09-24 00:56:00  107.492  107.494  107.486  107.486     9.0      True
2019-09-24 00:57:00  107.484  107.486  107.478  107.478     7.0      True
2019-09-24 00:58:00  107.480  107.482  107.473  107.478     8.0      True
2019-09-24 00:59:00  107.476  107.480  107.473  107.480     7.0      True
2019-09-24 01:00:00  107.478  107.488  107.478  107.486     6.0      True

[5000 rows x 6 columns]
             

                        open     high      low    close  amount  complete
time                                                                     
2019-11-04 19:01:00  108.376  108.378  108.372  108.378    10.0      True
2019-11-04 19:02:00  108.381  108.381  108.376  108.376     5.0      True
2019-11-04 19:03:00  108.378  108.388  108.378  108.385     9.0      True
2019-11-04 19:04:00  108.383  108.395  108.383  108.392     7.0      True
2019-11-04 19:05:00  108.395  108.403  108.393  108.398    10.0      True
...                      ...      ...      ...      ...     ...       ...
2019-11-08 11:59:00  109.223  109.225  109.218  109.224    10.0      True
2019-11-08 12:00:00  109.227  109.234  109.227  109.229     8.0      True
2019-11-08 12:01:00  109.226  109.231  109.224  109.228     6.0      True
2019-11-08 12:02:00  109.225  109.225  109.222  109.222     2.0      True
2019-11-08 12:03:00  109.224  109.227  109.222  109.222     8.0      True

[5000 rows x 6 columns]
             

                        open     high      low    close  amount  complete
time                                                                     
2019-12-21 00:34:00  109.378  109.378  109.373  109.373     3.0      True
2019-12-21 00:35:00  109.370  109.372  109.364  109.366     7.0      True
2019-12-21 00:36:00  109.364  109.364  109.361  109.361     2.0      True
2019-12-21 00:37:00  109.358  109.386  109.358  109.374    19.0      True
2019-12-21 00:38:00  109.376  109.376  109.372  109.374     4.0      True
...                      ...      ...      ...      ...     ...       ...
2019-12-30 10:18:00  109.384  109.384  109.378  109.381     6.0      True
2019-12-30 10:19:00  109.383  109.383  109.374  109.374     7.0      True
2019-12-30 10:20:00  109.376  109.376  109.368  109.368    12.0      True
2019-12-30 10:21:00  109.372  109.372  109.369  109.370     4.0      True
2019-12-30 10:22:00  109.366  109.366  109.359  109.364     8.0      True

[5000 rows x 6 columns]
             

,open,high,low,close,amount,complete
time,,,,,,
2019-01-28 09:00:00,109.416,109.430,109.382,109.412,45.0,True
2019-01-28 09:01:00,109.414,109.428,109.400,109.428,23.0,True
2019-01-28 09:02:00,109.426,109.426,109.396,109.419,30.0,True
2019-01-28 09:03:00,109.422,109.424,109.420,109.424,10.0,True
2019-01-28 09:04:00,109.422,109.424,109.414,109.419,8.0,True
...,...,...,...,...,...,...
2020-01-28 09:29:00,108.942,108.947,108.941,108.947,11.0,True
2020-01-28 09:30:00,108.946,108.953,108.946,108.950,29.0,True
2020-01-28 09:31:00,108.948,108.956,108.948,108.948,21.0,True


In [14]:
df = df.reset_index().drop_duplicates(subset='time',keep='first').set_index('time')
df

,open,high,low,close,amount,complete
time,,,,,,
2019-01-28 09:00:00,109.416,109.430,109.382,109.412,45.0,True
2019-01-28 09:01:00,109.414,109.428,109.400,109.428,23.0,True
2019-01-28 09:02:00,109.426,109.426,109.396,109.419,30.0,True
2019-01-28 09:03:00,109.422,109.424,109.420,109.424,10.0,True
2019-01-28 09:04:00,109.422,109.424,109.414,109.419,8.0,True
...,...,...,...,...,...,...
2020-01-28 09:29:00,108.942,108.947,108.941,108.947,11.0,True
2020-01-28 09:30:00,108.946,108.953,108.946,108.950,29.0,True
2020-01-28 09:31:00,108.948,108.956,108.948,108.948,21.0,True


In [15]:
df.to_csv('20190128_090000_20200128_093300_M1.csv')